# Census Notebook

Held every 10 years, the US census gives a detailed snapshot in time about the makeup of the country.  The last census in 2010 surveyed nearly 309 million people.  IPUMS.org provides researchers an open source data set with 1% to 10% of the census data set.  In this notebook, we want to see how education affects total income earned in the US based on data from each census from the 1970 to 2010 and see if we can predict some results if the census was held today, according to the national average.  We will go through the ETL, training the model, and then testing the prediction.  We'll make every effort to get as balanced of a dataset as we can.  We'll also pull some extra variables to allow for further self-exploration of gender based education and income breakdowns.  On a single Titan RTX, you can run the whole notebook workflow on the 4GB dataset of 14 million rows by 44 columns in less than 3 minutes, however this Colab customized notebook lets you explore a subset of that data, 2GB, while avoiding Out Of Memory issues.  

**Let's begin!**

In [0]:
!wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
!bash rapids-colab.sh

import sys, os

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

## Imports

In [0]:
import pandas as pd
import numpy as np
import cuml
import cudf
import dask_cudf
import dask_cuml
import sys
import os
import gzip
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

## Get your data!

In [0]:
#### !!!!!! UNCOMMENT TO DOWNLOAD DATASET !!!!!! #####
#!wget https://rapidsai-data.s3.us-east-2.amazonaws.com/datasets/ipums+1970+1990+2000.csv.gz 

In [0]:
def load_data(cached = 'ipums+1970+1990+2000.csv.gz',source='ipums'):
    if os.path.exists(cached) and source=='ipums':
        print('use ipums data')
        with gzip.open(cached) as f:
            X = cudf.read_csv(f)
    else:
        print("No data found!  Please uncomment the cell above the 'LOAD_DATA' function and try again!")
        X = null
    return X

In [0]:
file = 'ipums+1970+1990+2000.csv.gz' #incase you wanted to change your file later
df = load_data(file)
print('data',df.shape)

In [0]:
print(df.head(5).to_pandas())

In [34]:
df.dtypes

YEAR             int64
DATANUM          int64
SERIAL           int64
HHWT             int64
CPI99          float64
GQ               int64
QGQ            float64
PERNUM           int64
PERWT            int64
SEX              int64
AGE              int64
EDUC             int64
EDUCD            int64
INCTOT           int64
SEX_HEAD       float64
SEX_MOM        float64
SEX_POP        float64
SEX_SP         float64
SEX_MOM2       float64
SEX_POP2       float64
AGE_HEAD       float64
AGE_MOM        float64
AGE_POP        float64
AGE_SP         float64
AGE_MOM2       float64
AGE_POP2       float64
EDUC_HEAD      float64
EDUC_MOM       float64
EDUC_POP       float64
EDUC_SP        float64
EDUC_MOM2      float64
EDUC_POP2      float64
EDUCD_HEAD     float64
EDUCD_MOM      float64
EDUCD_POP      float64
EDUCD_SP       float64
EDUCD_MOM2     float64
EDUCD_POP2     float64
INCTOT_HEAD    float64
INCTOT_MOM     float64
INCTOT_POP     float64
INCTOT_SP      float64
INCTOT_MOM2    float64
INCTOT_POP2

In [35]:
original_counts = df.YEAR.value_counts()
print(original_counts) ### Remember these numbers!

2000    2808457
1990    2479020
1970    2029633
dtype: int64


## ETL

### Cleaning Income data
First, let's focus on cleaning out the bad values for Total Income `INCTOT`. First, let's see if there are an `N/A` values, as when we did `head()`, we saw some in other columns, like CBSERIAL

In [0]:
df['INCTOT_NA'] = df['INCTOT'].isna()

In [11]:
print(df.INCTOT_NA.value_counts())

False    7317110
dtype: int64


Okay, great, there are no `N/A`s...or are there?  Let's drop `INCTOT_NA` and see what our value counts look like

In [12]:
df=df.drop('INCTOT_NA')
print(df.INCTOT.value_counts().to_pandas())  ### Wow, look how many people in America make $10,000,000!  Wait a minutes... 

9999999    1724341
0           842576
20000        62816
30000        58371
12000        58248
10000        56314
15000        52982
25000        50034
6000         46820
18000        42550
40000        37964
5000         37637
8000         35022
35000        33098
24000        32471
16000        31692
14000        30682
22000        30501
13000        30469
5050         30458
3000         29963
50000        29820
3050         28882
17000        27928
7000         27675
9000         27451
6050         27190
4000         27159
4050         26786
28000        26663
            ...   
655950           1
657000           1
662000           1
666000           1
668500           1
669000           1
689000           1
695000           1
697000           1
705500           1
706000           1
708300           1
714000           1
719000           1
720400           1
722200           1
732000           1
736700           1
743000           1
757000           1
757100           1
762000      

Not that many people make $10M a year. Checking https://usa.ipums.org/usa-action/variables/INCTOT#codes_section, `9999999`is INCTOT's code for `N/A`.  That was why when we ran `isna`, RAPIDS won't find any.  Let's first create a new dataframe that is only NA values, then let's pull those encoded `N/A`s out of our working dataframe!

In [13]:
print('data',df.shape)
tdf = df.query('INCTOT == 9999999')
df = df.query('INCTOT != 9999999')

data (7317110, 44)


In [14]:
print('working data',df.shape)
print('junk count data',tdf.shape)

working data (5592769, 44)
junk count data (1724341, 44)


For the curious, now we should be able to get accurate Total Income data, by year, not taking into account inflation

In [15]:
print(df.groupby('YEAR')['INCTOT'].mean()) # without that cleanup, the average would have bene in the millions....

YEAR
1970    4189.597435738769
1990    18026.86175960633
2000    27395.80096278854
Name: INCTOT, dtype: float64


#### Normalize Income for inflation
Now that we have reduced our dataframe to a baseline clean data to answer our question, we should normalize the amounts for inflation.  `CPI99`is the value that IPUMS uses to contian the inflation factor.  All we have to do is multipy by year.  Let's see how that changes the Total Income values from just above!

In [16]:
print(df.groupby('YEAR')['CPI99'].mean()) ## it just returns the CPI99
df['INCTOT'] = df['INCTOT'] * df['CPI99']
print(df.groupby('YEAR')['INCTOT'].mean()) ## let's see what we got!

YEAR
1970     4.540000000033649
1990    1.3440000000313372
2000                   1.0
Name: CPI99, dtype: float64
YEAR
1970     19020.77235825401
1990    24228.102204913546
2000     27395.80096278854
Name: INCTOT, dtype: float64


### Cleaning Education Data
Okay, great!  Now we have income cleaned up, it should also have cleaned much of our next sets of values of interes, namely Education and Education Detailed.  However, there are still some `N/A`s in key variables to worry about, which can cause problmes later.  Let's create a list of them...

In [0]:
suspect = ['EDUC', 'EDUCD', 'EDUC_HEAD', 'EDUC_POP', 'EDUC_MOM','EDUCD_MOM2','EDUCD_POP2', 'INCTOT_MOM','INCTOT_POP','INCTOT_MOM2','INCTOT_POP2', 'INCTOT_HEAD']

In [19]:
for i in range(0, len(suspect)):
    df[suspect[i]] = df[suspect[i]].fillna(-1)
    print(suspect[i], df[suspect[i]].value_counts())

EDUC 6    1913252
7     766453
10     601171
2     587661
4     330352
11     329543
5     303339
8     302091
3     270371
1      86815
[2 more rows]
dtype: int64
EDUCD 62    1155015
71     696359
101     523720
60     401792
40     330352
50     303339
30     270371
65     196000
26     185359
114     182873
[27 more rows]
dtype: int64
EDUC_HEAD 6.0    1742489
7.0     715847
10.0     653443
2.0     580049
11.0     433226
-1.0     326618
8.0     299042
4.0     239050
5.0     206678
3.0     195514
[3 more rows]
dtype: int64
EDUC_POP -1.0    4958160
6.0     206684
2.0      87679
7.0      70045
10.0      64837
11.0      48976
8.0      33418
4.0      32807
5.0      27221
3.0      26886
[3 more rows]
dtype: int64
EDUC_MOM -1.0    4767173
6.0     325469
2.0     104471
7.0      91144
10.0      61419
4.0      46748
8.0      45503
5.0      40945
3.0      36593
11.0      30255
[3 more rows]
dtype: int64
EDUCD_MOM2 -1.0    5591766
62.0        291
71.0        142
101.0        106
81.0         56


Let's get drop any rows of any `-1`s in Education and Education Detailed.

In [20]:
totincome = ['EDUC','EDUCD']
for i in range(0, len(totincome)):
    query = totincome[i] + ' != -1'
    df = df.query(query)
    print(totincome[i])

EDUC
EDUCD


In [21]:
print(df.shape)
df.head().to_pandas().head()

(5592769, 44)


,YEAR,DATANUM,SERIAL,HHWT,CPI99,GQ,QGQ,PERNUM,PERWT,SEX,AGE,EDUC,EDUCD,INCTOT,SEX_HEAD,SEX_MOM,SEX_POP,SEX_SP,SEX_MOM2,SEX_POP2,AGE_HEAD,AGE_MOM,AGE_POP,AGE_SP,AGE_MOM2,AGE_POP2,EDUC_HEAD,EDUC_MOM,EDUC_POP,EDUC_SP,EDUC_MOM2,EDUC_POP2,EDUCD_HEAD,EDUCD_MOM,EDUCD_POP,EDUCD_SP,EDUCD_MOM2,EDUCD_POP2,INCTOT_HEAD,INCTOT_MOM,INCTOT_POP,INCTOT_SP,INCTOT_MOM2,INCTOT_POP2
0,1970,2,1,100,4.54,1,0.0,1,100,1,39,6,60,56523.0,1.0,NaN,NaN,2.0,NaN,NaN,39.0,NaN,NaN,36.0,NaN,NaN,6.0,-1.0,-1.0,3.0,NaN,NaN,60.0,NaN,NaN,30.0,-1.0,-1.0,12450.0,-1.0,-1.0,3450.0,-1.0,-1.0
1,1970,2,1,100,4.54,1,0.0,2,100,2,36,3,30,15663.0,1.0,NaN,NaN,1.0,NaN,NaN,39.0,NaN,NaN,39.0,NaN,NaN,6.0,-1.0,-1.0,6.0,NaN,NaN,60.0,NaN,NaN,60.0,-1.0,-1.0,12450.0,-1.0,-1.0,12450.0,-1.0,-1.0
2,1970,2,2,100,4.54,1,0.0,1,100,1,56,7,70,41087.0,1.0,NaN,NaN,2.0,NaN,NaN,56.0,NaN,NaN,54.0,NaN,NaN,7.0,-1.0,-1.0,6.0,NaN,NaN,70.0,NaN,NaN,60.0,-1.0,-1.0,9050.0,-1.0,-1.0,0.0,-1.0,-1.0
3,1970,2,2,100,4.54,1,0.0,2,100,2,54,6,60,0.0,1.0,NaN,NaN,1.0,NaN,NaN,56.0,NaN,NaN,56.0,NaN,NaN,7.0,-1.0,-1.0,7.0,NaN,NaN,70.0,NaN,NaN,70.0,-1.0,-1.0,9050.0,-1.0,-1.0,9050.0,-1.0,-1.0
4,1970,2,4,100,4.54,1,0.0,1,100,1,82,1,17,33823.0,1.0,NaN,NaN,2.0,NaN,NaN,82.0,NaN,NaN,74.0,NaN,NaN,1.0,-1.0,-1.0,2.0,NaN,NaN,17.0,NaN,NaN,23.0,-1.0,-1.0,7450.0,-1.0,-1.0,650.0,-1.0,-1.0


Well, the good news is that we lost no further rows, start to normalize the data so when we do our OLS, one year doesn't unfairly dominate the data

## Normalize the Data
The in the last step, need to keep our data at about the same ratio as we when started (1% of the population), with the exception of 1980, which was a 5% and needs to be reduced.  This is why we kept the temp dataframe `tdf` - to get the counts per year.   we will find out just how many have to realize

In [22]:
print('Working data: \n', df.YEAR.value_counts())
print('junk count data: \n', tdf.YEAR.value_counts())

Working data: 
 2000    2199860
1990    1906165
1970    1486744
dtype: int64
junk count data: 
 2000    608597
1990    572855
1970    542889
dtype: int64


And now, so that we can do MSE, let's make all the dtypes the same. 

In [23]:
df.dtypes

YEAR             int64
DATANUM          int64
SERIAL           int64
HHWT             int64
CPI99          float64
GQ               int64
QGQ            float64
PERNUM           int64
PERWT            int64
SEX              int64
AGE              int64
EDUC             int64
EDUCD            int64
INCTOT         float64
SEX_HEAD       float64
SEX_MOM        float64
SEX_POP        float64
SEX_SP         float64
SEX_MOM2       float64
SEX_POP2       float64
AGE_HEAD       float64
AGE_MOM        float64
AGE_POP        float64
AGE_SP         float64
AGE_MOM2       float64
AGE_POP2       float64
EDUC_HEAD      float64
EDUC_MOM       float64
EDUC_POP       float64
EDUC_SP        float64
EDUC_MOM2      float64
EDUC_POP2      float64
EDUCD_HEAD     float64
EDUCD_MOM      float64
EDUCD_POP      float64
EDUCD_SP       float64
EDUCD_MOM2     float64
EDUCD_POP2     float64
INCTOT_HEAD    float64
INCTOT_MOM     float64
INCTOT_POP     float64
INCTOT_SP      float64
INCTOT_MOM2    float64
INCTOT_POP2

In [24]:

keep_cols = ['YEAR', 'DATANUM', 'SERIAL', 'HHWT', 'GQ', 'PERNUM', 'SEX', 'AGE', 'INCTOT', 'EDUC', 'EDUCD', 'EDUC_HEAD', 'EDUC_POP', 'EDUC_MOM','EDUCD_MOM2','EDUCD_POP2', 'INCTOT_MOM','INCTOT_POP','INCTOT_MOM2','INCTOT_POP2', 'INCTOT_HEAD', 'SEX_HEAD']
df = df.loc[:, keep_cols]
#df = df.drop(col for col in df.columns if col not in keep_cols)
for i in range(0, len(keep_cols)):
    df[keep_cols[i]] = df[keep_cols[i]].fillna(-1)
    print(keep_cols[i], df[keep_cols[i]].value_counts())
    df[keep_cols[i]]= df[keep_cols[i]].astype('float64')

YEAR 2000    2199860
1990    1906165
1970    1486744
dtype: int64
DATANUM 4    4106025
2    1486744
dtype: int64
SERIAL 516649    27
274893    24
1032196    22
502165    21
546122    21
740629    21
50243    20
95951    20
528890    20
129098    19
[1223216 more rows]
dtype: int64
HHWT 100    5592769
dtype: int64
GQ 1    5385260
4     103757
3      93699
2       9529
5        522
0          2
dtype: int64
PERNUM 1    2804172
2    1858027
3     547105
4     221665
5      87826
6      38633
7      17909
8       7815
9       4189
10       2390
[13 more rows]
dtype: int64
SEX 2    2916655
1    2676114
dtype: int64
AGE 19    114873
18    112782
17    112483
16    112309
20    112074
30    111875
29    109707
40    109570
35    109055
27    108716
[77 more rows]
dtype: int64
INCTOT 0.0    842576
30000.0     35437
20000.0     34016
22927.0     30173
16128.000000000002     29344
13440.0     28944
12000.0     28904
26880.0     28819
25000.0     28702
13847.0     28635
[94807 more rows]
dtype: i

In [0]:
## I WANTED TO REDUCE THE 1980 SAMPLE HERE, BUT .SAMPLE() IS NEEDED AND NOT WORKING, UNLESS THERE IS A WORK AROUND...

With the important data now clean and normalized, let's start doing the regression

## Ridge Regression
We have 44 variables.  The other variables may provide important predictive information.  The Ridge Regression technique with cross validation to identify the best hyperparamters may be the best way to get the most accurate model.  We'll have to 

* define our performance metrics
* split our data into train and test sets
* train and test our model

Let's begin and see what we get!

In [0]:
# As our performance metrics we'll use a basic mean squared error and coefficient of determination implementation
def mse(y_test, y_pred):
    return ((y_test - y_pred) ** 2).mean()

def cod(y_test, y_pred):
    y_bar = y_test.mean()
    total = ((y_test - y_bar) ** 2).sum()
    residuals = ((y_test - y_pred) ** 2).sum()
    return 1 - (residuals / total)

In [0]:
from cuml.preprocessing.model_selection import train_test_split
trainsize = .9
yCol = "EDUC"
from cuml.preprocessing.model_selection import train_test_split
from cuml.linear_model.ridge import Ridge

def train_and_score(data, clf, train_frac=0.8, n_runs=20):
    mse_scores, cod_scores = [], []
    for _ in range(n_runs):
        X_train, X_test, y_train, y_test = cuml.preprocessing.model_selection.train_test_split(df, yCol, train_size=.9)
        y_pred = clf.fit(X_train, y_train).predict(X_test)
        mse_scores.append(mse(y_test, y_pred))
        cod_scores.append(cod(y_test, y_pred))
    return mse_scores, cod_scores

 ## Results
 **Moment of truth!  Let's see how our regression training does!**

In [28]:
import numpy as np
n_runs = 20
clf = Ridge()
mse_scores, cod_scores = train_and_score(df, clf, n_runs=n_runs)
print(f"median MSE ({n_runs} runs): {np.median(mse_scores)}")
print(f"median COD ({n_runs} runs): {np.median(cod_scores)}")

median MSE (20 runs): nan
median COD (20 runs): 1.0


**Fun fact:** if you made INCTOT the y axis, your prediction results would not be so pretty!  It just shows that your education level can be an indicator for your income, but your income is NOT a great predictor for your education level.  You have better odds flipping a coin!

* median MSE (50 runs): 518189521.07548225
* median COD (50 runs): 0.425769113846303

## Next Steps/Self Study
* You can pickle the model and use it in another workflow
* You can redo the workflow with based on head of household using `EDUC`, `SEX`, and `INCTOT` for X in `X`_HEAD
* You can see the growing role of education with women in their changing role in the workforce and income with "EDUC_MOM" and "EDUC_POP